In [14]:
# This script extracts the title, link, and short description of search results on Google 

#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import requests
import re


In [2]:
# input for keyword, number of pages to scrape, and website URL 
# only want top result so only search for 1 result 
nPages = 1
maxResult = 1
googleurl = 'https://www.google.com/'
siteURL = 'https://www.babylonhealth.com/en-us'
websiteName = 'Bablylon Health'
useURL = False
useName = False
calculateLiklihood = True 

# Booleans
if (siteURL != None):
    useURL = True; 
    useName = False; 
else:
    useURL = False; 
    useName = True; 
    
# format queries based on inputs 
#queryURL = "site: " + siteURL
#queryName = websiteName
#print('useURL: ' + str(useURL))
#print('useName: ' + str(useName))

In [3]:
# Access chromedriver and determine path 
driver = webdriver.Chrome(ChromeDriverManager().install())
pathToFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Deliverables/'
driver.get(googleurl)




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/zacharywong/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache
/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_72549/2887605795.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# set the keyword you want to search for depending on whether link or name is given
# we find the search bar using it's name attribute value
searchBar = driver.find_element_by_name('q')


# first we send our keyword to the search bar followed by the enter # key depending on using URL or website name 

if (useURL):
    query = "site: " + siteURL
    searchBar.send_keys(query)
    searchBar.send_keys('\n')
else:
    query = websiteName
    searchBar.send_keys(query)
    searchBar.send_keys('\n')

/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_72549/331980782.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  searchBar = driver.find_element_by_name('q')


In [5]:
# list of dictionaries with key/value pairs: title, link, text
# Contains all information for all search results 
pageInfo = []

In [6]:
# helper function to extract links from 1 page of results
def extractLinks(soup):
    links = []
    searchLinks = soup.find_all('div', class_ = 'yuRUbf')
    for h in searchLinks:
        link = h.a.get('href')
        links.append(link)
    return links 

In [7]:
# helper function to extract texts from 1 page of results
def extractTexts(soup):
    texts = []
    searchText = soup.find_all('div', class_='VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf')
    for h in searchText:
        fullText = h.text
        try:
            splitText = fullText.split('— ', 1)
            text = splitText[1]
            texts.append(text);
        except:
            texts.append(fullText)
    return texts;

In [8]:
# helper function to extract titles from 1 page of results
def extractTitles(soup):
    titles = [] 
    searchTitles = soup.find_all('h3', class_='LC20lb MBeuO DKV0Md')
    for h in searchTitles:
        titles.append(h.text)
    return titles

In [ ]:
def extractEmails(soup):
    allLinks = []
    searchLinks = soup.find_all()

In [9]:
def calculateLiklihoods(links):
    keywords = ['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', '24/7', 'democratizing']
    liklihoods = []
    detectedWordsAll = []
    for url in links: 
        detectedWords = []
        output = ''
        count = 0
        liklihood = 0

        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_page = res.content
        soup = bs(html_page, 'html.parser')
        text = soup.find_all(text=True)

        #ignore = ['[document]', 'a', 'article', 'label','div','script', 'style', 'title', 'img', 'svg', 'ul', 'g', 'footer', 'button', 'clippath', 'nav']
        ignore1 = ['[document]', 'label','div','script', 'style', 'img', 'svg', 'ul', 'g', 'footer', 'button', 'clippath', 'nav']
       
        for t in text:
            if t.parent.name not in ignore1:
                output += '{} '.format(t)

        # analyze the webpage to detect keywords 
        outputSub = output.split(' ')
        #print(outputSub)
        for word in keywords: 
            if (word in outputSub or word.capitalize() in outputSub):
                #print('"' + word +'"' + ' detected')
                detectedWords.append(word)
                count +=1 
            #else:
                #print(word + ' is not there')
        liklihood = round((count / len(keywords)), 2)
        liklihoods.append(liklihood)
        detectedWordsAll.append(detectedWords)
    return liklihoods, detectedWordsAll

In [10]:
# helper function to add 1 page of results to pageInfo list
def addToPageInfo(titles, links, texts, liklihoods, detectedWordsAll):
    index = 0;
    while (index < maxResult):
        # create new dictionary of each search results' title, link, and text
        if (liklihoods == None):
            pageInfo.append({"Title": titles[index], "Link": links[index], "About": texts[index]})
        else:
            pageInfo.append({"Liklihood": liklihoods[index], "Title": titles[index], "Link": links[index], "About": texts[index],  "DetectedWords": detectedWordsAll[index]})
        index += 1

    

In [11]:
#capture links, header, and text
#pageInfo is a list of dictionaries for each page with keys/value pairs: header, link, text
# extract and load each page of results to pageInfo 

for page in range(1, nPages+1):
    soup = bs(driver.page_source, 'html.parser')
    links = extractLinks(soup);
    texts = extractTexts(soup);
    titles = extractTitles(soup);
    emails = extractEmails(soup); 
    if (calculateLiklihood):
        liklihoods, detectedWordsAll = calculateLiklihoods(links)
        addToPageInfo(titles, links, texts, liklihoods, detectedWordsAll)
    else:
        addToPageInfo(titles, links, texts, liklihoods = None, detectedWordsAll = None)
    
pageInfo 

        
    

[{'Liklihood': 0.43,
  'Title': 'More than just a doctor on your phone - Babylon Health',
  'Link': 'https://www.babylonhealth.com/en-us',
  'About': 'Healthcare anytime, from your phone. Book a Babylon Virtual Doctor Appointment 24/7, check your symptoms with our artificial intelligence powered app.',
  'DetectedWords': ['personalized', 'AI', '24/7']}]

In [12]:
# convert pageInfo to pandas dataframe and export as csv 
df = pd.DataFrame(pageInfo)
df.to_csv(pathToFile + 'Info_' + websiteName + '.csv')

valid email
